# Spectral Sounds: Unveiling Music Genre Classification Through Audio Spectrograms

# Introduction:
With the dominance of digital music streaming, music genre classification is an increasingly important way to develop algorithms for suggesting new songs to users. Without a way to analyze the abstract aspects of each song such as mood and tone, these algorithms must be able to make accurate predictions based only on the quantitative characteristics of the audio itself.

Hence, we aim to answer the following research question:
 **Can we predict the genre of a song based on its audio spectrograph information?** In particular, we will focus on the **classical, jazz, pop, and rock music** genres.

### Dataset Description
We will be using the Music Genre Classification dataset by Andrada on [Kaggle](https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification). It includes observations for 1000 30-second audio snippets (.wav ), with 60 relevant variables that describe mean and variance values of different aspects of its audio spectrogram (visual waveform) labeled with the corresponding genre of music. Due to the number of predictor variables, we only choose a few potentially important predictors to focus on for the preliminary analysis (explanation in [Methods](#Methods) section).


## Preliminary Data Analysis

### Setting Up Libraries and Parameters

In [5]:
# Run this first.
library(tidyverse)
library(tidymodels)
library(RColorBrewer)
library(cowplot)
set.seed(45)

DATASET_URL <- "https://raw.githubusercontent.com/arthu-rguo/dsci-100-proj/master/features_30_sec.csv"
GENRES <- c("classical", "jazz", "pop", "rock")
VARIABLES <- c("chroma_stft_mean", "rms_mean", "spectral_centroid_mean", "spectral_bandwidth_mean", "zero_crossing_rate_mean", "chroma_stft_var", "rms_var", "spectral_centroid_var", "spectral_bandwidth_var", "zero_crossing_rate_var")
# "chroma_stft_var", "rms_var", "spectral_centroid_var", "spectral_bandwidth_var", "zero_crossing_rate_var

### Loading and Tidying Data

In [6]:
# Load and tidy up the dataset.
music_data_raw <- read_csv(DATASET_URL)

music_data <- music_data_raw |>
    select(label, all_of(VARIABLES)) |>
    filter(label %in% GENRES) |> 
    mutate(label = as.factor(label)) |>
    relocate(label)

head(music_data)

# Split the dataset into training and testing sets.
music_split <- music_data |> 
    initial_split(props = 0.75, strata = label)

music_training_data <- training(music_split)
music_testing_data <- testing(music_split)

Rows: 1000 Columns: 60
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): filename, label
dbl (58): length, chroma_stft_mean, chroma_stft_var, rms_mean, rms_var, spec...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


label,chroma_stft_mean,rms_mean,spectral_centroid_mean,spectral_bandwidth_mean,zero_crossing_rate_mean,chroma_stft_var,rms_var,spectral_centroid_var,spectral_bandwidth_var,zero_crossing_rate_var
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
classical,0.2523919,0.03624489,1505.166,1559.006,0.09822270,0.08415318,0.0002207507,44457.89,43682.97,0.0004313938
classical,0.2149950,0.03062517,1360.644,1441.243,0.08776563,0.08500956,0.0001609684,61611.21,32378.73,0.0006296792
classical,0.2779581,0.04385955,1490.275,1600.116,0.09004578,0.08375870,0.0001926202,38993.34,31200.90,0.0003952271
classical,0.2342293,0.02944208,1526.630,1499.439,0.10894638,0.08345093,0.0003565407,91344.31,60196.23,0.0011608430
classical,0.2215738,0.04231631,1705.563,1646.064,0.11285717,0.08499950,0.0002093876,68132.45,25963.46,0.0008350001
classical,0.2685303,0.04326224,1431.055,1558.113,0.07521004,0.08254110,0.0005845473,74049.94,39297.27,0.0006178058


In [7]:
#this is predictor selection
# create an empty tibble to store the results
names <- colnames(music_training_data |> select(-label))
accuracies <- tibble(size = integer(), 
                     model_string = character(), 
                     accuracy = numeric())

# create a model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) |>
     set_engine("kknn") |>
     set_mode("classification")

# create a 5-fold cross-validation object
music_vfold <- vfold_cv(music_training_data, v = 5, strata = label)

# store the total number of predictors
n_total <- length(names)

# stores selected predictors
selected <- c()

# for every size from 1 to the total number of predictors
for (i in 1:n_total) {
    # for every predictor still not added yet
    accs <- list()
    models <- list()
    for (j in 1:length(names)) {
        # create a model string for this combination of predictors
        preds_new <- c(selected, names[[j]])
        model_string <- paste("label", "~", paste(preds_new, collapse="+"))

        # create a recipe from the model string
        music_recipe <- recipe(as.formula(model_string), 
                                data = music_training_data) |>
                          step_scale(all_predictors()) |>
                          step_center(all_predictors())

        # tune the KNN classifier with these predictors, 
        # and collect the accuracy for the best K
        acc <- workflow() |>
          add_recipe(music_recipe) |>
          add_model(knn_spec) |>
          tune_grid(resamples = music_vfold, grid = 10) |>
          collect_metrics() |>
          filter(.metric == "accuracy") |>
          summarize(mx = max(mean))
        acc <- acc$mx |> unlist()

        # add this result to the dataframe
        accs[[j]] <- acc
        models[[j]] <- model_string
    }
    jstar <- which.max(unlist(accs))
    accuracies <- accuracies |> 
      add_row(size = i, 
              model_string = models[[jstar]], 
              accuracy = accs[[jstar]])
    selected <- c(selected, names[[jstar]])
    names <- names[-jstar]
}
accuracies

size,model_string,accuracy
<int>,<chr>,<dbl>
1,label ~ rms_mean,0.5900000
2,label ~ rms_mean+spectral_bandwidth_mean,0.7300000
3,label ~ rms_mean+spectral_bandwidth_mean+chroma_stft_mean,0.7733333
4,label ~ rms_mean+spectral_bandwidth_mean+chroma_stft_mean+zero_crossing_rate_mean,0.8033333
5,label ~ rms_mean+spectral_bandwidth_mean+chroma_stft_mean+zero_crossing_rate_mean+rms_var,0.8133333
6,label ~ rms_mean+spectral_bandwidth_mean+chroma_stft_mean+zero_crossing_rate_mean+rms_var+zero_crossing_rate_var,0.8300000
7,label ~ rms_mean+spectral_bandwidth_mean+chroma_stft_mean+zero_crossing_rate_mean+rms_var+zero_crossing_rate_var+spectral_centroid_var,0.8166667
8,label ~ rms_mean+spectral_bandwidth_mean+chroma_stft_mean+zero_crossing_rate_mean+rms_var+zero_crossing_rate_var+spectral_centroid_var+spectral_centroid_mean,0.8033333
9,label ~ rms_mean+spectral_bandwidth_mean+chroma_stft_mean+zero_crossing_rate_mean+rms_var+zero_crossing_rate_var+spectral_centroid_var+spectral_centroid_mean+chroma_stft_var,0.8033333


### Data Summary (of Training Set)
The following table displays a count of each genre of music we are focusing on, as well as the median of the 5 audio characterisics in each group. 

In [ ]:
# Summarize the training dataset in a table.
music_summary <- music_training_data |>
    group_by(label) |> 
    summarize(count = n(), across(everything(), median))

music_summary

### Preliminary Data Visualization
We examine the disribution of 5 mean audio characteristics for the chosen genres of music.

In [ ]:
# Visualize the training dataset with a plot.
options(repr.plot.width = 12, repr.plot.height = 7)

music_summary_labels <- music_summary |>
    select(label)

music_summary_norms <- music_summary |>
    select(all_of(VARIABLES)) |>
    mutate(across(everything(), scale)) |>
    mutate(across(everything(), abs))

music_summary_long <- bind_cols(music_summary_labels, music_summary_norms) |>
    pivot_longer(cols = -label, names_to = "name", values_to = "value")

music_visualization <- music_summary_long |>
    ggplot(aes(x = name, y = value, fill = name)) +
        geom_bar(stat = "identity") +
        facet_wrap(~label, ncol = 2) +
        labs(x = "Audio Characteristics", y = "Value (normalized)", title = "Distribution of Audio Characteristics per Genre") +
        theme(legend.pos = "none", plot.title = element_text(size = 20), text = element_text(size = 14)) +
        scale_fill_brewer(palette = "Set1") +
        coord_flip()

music_visualization

We also visualize a few potential grouping pattern that might occur between different predictor variables.

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 6)

template <- music_training_data |>
            ggplot(aes(x = zero_crossing_rate_mean, y = spectral_bandwidth_mean, color = label)) +
                geom_point() +
                theme(plot.title = element_text(size = 20), text = element_text(size = 14)) +
                labs(x = "Zero Crossing Rate (crossings/byte)", y = "Spectral Bandwidth (Hz)", color = "Music Genres")

zc_vs_sb <- music_training_data |>
            ggplot(aes(x = zero_crossing_rate_mean, y = spectral_bandwidth_mean, color = label)) +
                geom_point() +
                theme(legend.pos = "none", plot.title = element_text(size = 20), text = element_text(size = 14)) +
                labs(x = "Zero Crossing Rate (crossings/byte)", y = "Spectral Bandwidth (Hz)")

rms_vs_sb <- music_training_data |>
            ggplot(aes(x = rms_mean, y = spectral_bandwidth_mean, color = label)) +
                geom_point() +
                theme(legend.pos = "none", plot.title = element_text(size = 20), text = element_text(size = 14)) +
                labs(x = "Root Mean Square (dB)", y = "")

rms_vs_zc <- music_training_data |>
            ggplot(aes(y = rms_mean, x = zero_crossing_rate_mean, color = label)) +
                geom_point() +
                theme(legend.pos = "none", plot.title = element_text(size = 20), text = element_text(size = 14)) +
                labs(y = "Root Mean Square (dB)", x = "Zero Crossing Rate (crossings/byte)")

rms_vs_sc <- music_training_data |>
            ggplot(aes(x = spectral_centroid_mean, y = rms_mean, color = label)) +
                geom_point() +
                theme(legend.pos = "none", plot.title = element_text(size = 20), text = element_text(size = 14)) +
                labs(x = "Spectral Centroid (Hz)", y = "")

legend <- get_legend(
  # create some space to the left of the legend
  template + theme(legend.box.margin = margin(0, 0, 0, 12))
)

options(repr.plot.width = 12, repr.plot.height = 9)

plots <- plot_grid(zc_vs_sb, rms_vs_sb, rms_vs_zc, rms_vs_sc,
                ncol = 2, nrow = 2)

final_plot <- plot_grid(plots, legend, rel_widths = c(3, .4))

final_plot

## Methods

To classify song genre based on audio features, the data will undergo preprocessing, feature extraction, data exploration, data splitting, model training using the $K$-nearest-neighbors algorithm, model evaluation, 5-fold cross validation and model optimization. To exteact the relevant features from the audio data for model training, we will tune $K$ values as well as performing iterative perdictor variable selection. Finally, we will evaluate its performance against a test set, interpret, and report on the ability to classify genre based on audio features. 

A few useful predictor variables:

1. **Chroma feature:** Measures the distribution of musical pitch, and captures tonal content. Helpful in distinguishing genres that have distinct harmonic patterns.

2. **RMS (Root Mean Square) values:** Measures the perceived loudness of the signal. Useful in differentiating genres with varying loudness characteristics.

3. **Spectral centroid:** easures the average frequency of the signal. Aids in identifying genres that tend to have specific frequency characteristics.

4. **Spectral bandwidth:** Measures the average spread of frequencies of the signal. Helps differentiate genres with varying spectral width or frequency spread.

5. **Zero crossing rate:** Provides information about the signal's fluctuations. Useful in distinguishing genres with different levels of rhythmic complexity or smoothness.


### Visualization
- **Scatter plot:** Plot the audio features of songs as data points on a graph to identify clusters or patterns.
- **Heatmap:** Visualize the correlation between audio features using a matrix of colors.
- **Box plot:** Illustrate the distribution of audio features within genres using boxes and whiskers.

## Expected Outcomes and Significance

### Expected Findings
In general, the project anticipates identifying informative and discriminative features that strongly influence genre classification from the audio spectrograph, enabling a deeper understanding of the distinguishing characteristics of different music genres and providing valuable insights in genre classification based on audio features. 

### Potential Impacts
Overall, the findings can have practical implications in the music industry and related fields, including enhancing music discovery, improving musical analysis, and assisting music production processes. 

### Future Questions
- How does the performance of KNN classification compare to other algorithms for genre classification?

- How can genre classification models handle songs that contain elements of multiple genres or blur genre boundaries?